# Sumarização

Será utilizado como exemplo ações do mercado financeiro

## Pacotes

In [1]:
library(quantmod)
library(RCurl)
library(tidyr)
library(dplyr)
library(magrittr)
library(ggplot2)

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.
Loading required package: bitops

Attaching package: ‘tidyr’

The following object is masked from ‘package:RCurl’:

    complete


Attaching package: ‘dplyr’

The following objects are masked from ‘package:xts’:

    first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘magrittr’

The following object is masked from ‘package:tidyr’:

    extract



## Obtenção dos dados

### Fonte

In [2]:
sit <- getURLContent('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', binary=TRUE, followlocation = TRUE, ssl.verifypeer = FALSE)
con <- gzcon(rawConnection(sit, 'rb'))
source(con)
close(con)

### Busca os dados

In [5]:
tickers <- spl('^BVSP,PETR4.SA,VALE5.SA,USIM5.SA,ABEV3.SA,ITUB4.SA,BBDC4.SA,JBSS3.SA,GOLL4.SA')

dados <- new.env()
options( download.file.method="libcurl" )
getSymbols(tickers, src = 'yahoo', from = '2017-01-01', env = dados, auto.assign = T)

pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols
pausing 1 second between requests for more than 5 symbols


[1] "BVSP"     "PETR4.SA" "VALE5.SA" "USIM5.SA" "ABEV3.SA" "ITUB4.SA" "BBDC4.SA"
[8] "JBSS3.SA" "GOLL4.SA"

### Ajusta os dados

In [6]:
# Somente os precos
for(i in ls(dados)) dados[[i]] = adjustOHLC(dados[[i]], use.Adjusted=T)                            
bt.prep(dados, align = 'remove.na')

### Obtem somente os preços

In [7]:
dados_precos <- dados$prices
dados_precos <- data.frame(data = index(dados_precos), dados_precos)
head(dados_precos)

,data,ABEV3.SA,BBDC4.SA,BVSP,GOLL4.SA,ITUB4.SA,JBSS3.SA,PETR4.SA,USIM5.SA,VALE5.SA
2017-01-02,2017-01-02,16.24416,26.0969,59589,4.51,32.84536,11.30,14.66,4.08,22.07962
2017-01-03,2017-01-03,16.57283,27.2039,61814,4.88,34.23636,11.45,15.50,4.29,23.29714
2017-01-04,2017-01-04,16.44335,27.0317,61589,5.17,34.38330,11.45,15.50,4.55,22.85265
2017-01-05,2017-01-05,16.53299,27.3309,62071,5.23,34.85350,11.42,15.75,4.60,23.91557
2017-01-06,2017-01-06,16.39355,27.5032,61665,5.15,34.97105,11.50,15.66,4.47,23.29714
2017-01-09,2017-01-09,16.46327,27.5395,61700,5.23,35.13758,11.60,15.33,4.50,23.78995


### Obtem as variações

In [8]:
diff <- dados_precos %>% 
            select(-data) %>% 
            mutate_each( funs(c(.[1], diff(log(zoo(.))))) )
names(diff) <- paste0("var_", names(diff) )
diff <- data.frame( data = dados_precos$data, diff)
head(diff)

data,var_ABEV3.SA,var_BBDC4.SA,var_BVSP,var_GOLL4.SA,var_ITUB4.SA,var_JBSS3.SA,var_PETR4.SA,var_USIM5.SA,var_VALE5.SA
2017-01-02,16.244160000,26.096900000,5.958900e+04,4.51000000,32.845360000,11.300000000,14.660000000,4.080000000,22.07962000
2017-01-03,0.020031148,0.041543812,3.665888e-02,0.07884807,0.041477750,0.013187004,0.055717327,0.050189745,0.05367560
2017-01-04,-0.007843467,-0.006350094,-3.646593e-03,0.05772747,0.004282744,0.000000000,0.000000000,0.058840500,-0.01926352
2017-01-05,0.005436639,0.011007679,7.795608e-03,0.01153859,0.013582581,-0.002623526,0.016000341,0.010929071,0.04546263
2017-01-06,-0.008469814,0.006284431,-6.562382e-03,-0.01541456,0.003367014,0.006980831,-0.005730675,-0.028667895,-0.02619911
2017-01-09,0.004243874,0.001318976,5.674219e-04,0.01541456,0.004750637,0.008658063,-0.021297998,0.006688988,0.02093262


### Transforma tudo em data.frame

In [9]:
dados_precos %<>%
    filter( row_number() != 1 ) %>% 
    gather(acao, preco, -data)
   
diff %<>% 
    filter( row_number() != 1 ) %>% 
    gather(acao, preco, -data)

dados_precos <- data.frame(dados_precos, variacao = diff$preco )

head(dados_precos)


data,acao,preco,variacao
2017-01-03,ABEV3.SA,16.57283,0.020031148
2017-01-04,ABEV3.SA,16.44335,-0.007843467
2017-01-05,ABEV3.SA,16.53299,0.005436639
2017-01-06,ABEV3.SA,16.39355,-0.008469814
2017-01-09,ABEV3.SA,16.46327,0.004243874
2017-01-10,ABEV3.SA,16.27404,-0.011560639


## Sumarizar dados

In [10]:
dados_precos %>% 
    group_by(acao) %>% 
    summarize( ultimo_preco = last(preco),
               retorno = round( mean(variacao)*100, 4),
               risco = round( sd(variacao)*100, 4) ) %>% 
    arrange( desc(risco) )
    

acao,ultimo_preco,retorno,risco
GOLL4.SA,10.70,1.0409,3.7465
USIM5.SA,4.18,0.0292,3.2687
VALE5.SA,24.94,0.1468,3.2507
PETR4.SA,14.21,-0.0376,2.3066
BBDC4.SA,30.73,0.1969,2.2730
JBSS3.SA,10.70,-0.0657,2.2033
ITUB4.SA,38.70,0.1976,1.5362
ABEV3.SA,18.90,0.1824,1.2206
BVSP,65710.00,0.1178,1.2060


## Criar grupos dos dados sumarizados

In [11]:
dados_sumarizados <- dados_precos %>% 
                        group_by(acao) %>% 
                        summarize( ultimo_preco = last(preco),
                                   retorno = round( mean(variacao)*100, 4),
                                   risco = round( sd(variacao)*100, 4) ) %>% 
                        mutate(class_retorno = ifelse( retorno > 0.15, "ALTO", "BAIXO") ,
                               class_risco = ifelse( risco < 2, "BAIXO",
                                                ifelse( risco > 3, "ALTO", "MODERADO") ) )
dados_sumarizados

acao,ultimo_preco,retorno,risco,class_retorno,class_risco
ABEV3.SA,18.90,0.1824,1.2206,ALTO,BAIXO
BBDC4.SA,30.73,0.1969,2.2730,ALTO,MODERADO
BVSP,65710.00,0.1178,1.2060,BAIXO,BAIXO
GOLL4.SA,10.70,1.0409,3.7465,ALTO,ALTO
ITUB4.SA,38.70,0.1976,1.5362,ALTO,BAIXO
JBSS3.SA,10.70,-0.0657,2.2033,BAIXO,MODERADO
PETR4.SA,14.21,-0.0376,2.3066,BAIXO,MODERADO
USIM5.SA,4.18,0.0292,3.2687,BAIXO,ALTO
VALE5.SA,24.94,0.1468,3.2507,BAIXO,ALTO


## Tabelas cruzada classificação retorno e classificação de risco

In [20]:
print( "Frequencias Classificação Retorno" )
dados_sumarizados %>% 
    select(class_retorno) %>% 
    table()

print( "Frequencias Classificação Risco" )
dados_sumarizados %>% 
    select(class_risco) %>% 
    table()

print( "Tabela Cruzada Classificação Retorno e Risco" )
dados_sumarizados %>% 
    select(class_retorno, class_risco) %>% 
    table()


[1] "Frequencias Classificação Retorno"


.
 ALTO BAIXO 
    4     5 

[1] "Frequencias Classificação Risco"


.
    ALTO    BAIXO MODERADO 
       3        3        3 

[1] "Tabela Cruzada Classificação Retorno e Risco"


             class_risco
class_retorno ALTO BAIXO MODERADO
        ALTO     1     2        1
        BAIXO    2     1        2